# Task 2: Build your first AI agent

In this notebook, you learn how to build AI agents using the Strands Agents framework. You start by creating a conversational agent, then add tools to make it more capable. You finally build a practical recipe assistant that can search the web for cooking information. You use Amazon Bedrock with the Amazon Nova Lite model to power your agents.

AI agents differ from traditional LLMs because they can take actions, use tools, and work toward goals autonomously. Rather than only responding to questions, agents can actually do things for you.

#### Scenario
You work for AnyCompany, a growing technology company that wants to explore how AI agents can help automate everyday tasks. Your team is interested in learning how these agents could help with tasks like research and customer support. You are starting with the basics of building intelligent agents.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **Learn more:** As you work through this notebook, you can use Amazon Q Developer to help understand the Python code and logic. Amazon Q can explain code snippets, suggest improvements, and help you learn Python programming concepts.

**To access Amazon Q Developer in JupyterLab:**

<!-- ![Amazon Q location in JupyterLab](./images/amazon-q-location.png) -->
<img src="images/amazon-q-location.png" width=50% height=20% />

*Image description: The preceding screenshot shows the JupyterLab interface with the Amazon Q panel visible in the left sidebar. The Amazon Q icon and chat interface are highlighted, showing where students can access Amazon Q Developer features.*

1. Look for the **Amazon Q** panel in the left sidebar of your JupyterLab interface (as shown in the image above)
2. Once connected, you can:
   - Select a code block cell, right-click, and choose **Generative AI** > **Explain code**.
   - Type questions about Python syntax, functions, or programming concepts in the Amazon Q chat.
   - Ask for code suggestions or improvements.
   - Get help with debugging errors.

## Task 2.1: Environment setup

In this task, you set up your environment by installing the necessary packages to get started with building AI agents.

In [ ]:
# Install the Strands Agents framework and tools
%pip install strands-agents strands-agents-tools

## Task 2.2: Create your first AI agent

In this task, you create an AI agent that can have conversations. This agent uses Amazon Bedrock with the Amazon Nova Lite model to understand and respond to your messages. The system prompt defines the desired behavior of your agent.

In [12]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent

# Create your first AI agent
agent = Agent(callback_handler=None,
    model="us.amazon.nova-lite-v1:0",
    system_prompt="You are a helpful assistant that provides concise responses."
)

2025-12-08 15:00:30,175 | INFO | botocore.credentials | Found credentials in shared credentials file: ~/.aws/credentials


Next, you test your agent by sending it a message.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The Strands framework uses Amazon Bedrock with the Amazon Nova Lite model by default. This model supports conversational interactions and can be enhanced with tools.

In [13]:
# Send a message to the agent
response = agent("Hello! Tell me a joke.")
print(response)

Sure! Why did the scarecrow win an award? Because he was outstanding in his field!



## Task 2.3: Add tools to your agent

In this task, you give your agent tools so it can do more than chat.

- You add a calculator tool which is provided by the Strands Agents SDK.

- You create a weather tool using the @tool decorator. 

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The weather tool is a placeholder example and will always return ‘Sunny’

In [ ]:
from strands import Agent, tool
from strands_tools import calculator

# Create a weather tool
@tool
def weather():
    """Get current weather information"""
    return "Sunny and O degree Celsius"

# Create an agent with tools
agent_with_tools = Agent(callback_handler=None,
    model="us.amazon.nova-lite-v1:0",
    tools=[calculator, weather],
    system_prompt="You are a helpful assistant. You can do math calculations and use the weather tool to tell the weather."
)

# Test the agent with both tools in one query
# Note: This query requires both tools - the weather tool to get temperature in Celsius,
# and the calculator tool to convert from Celsius to Fahrenheit.
# The agent will automatically determine which tools to use and in what order.
response = agent_with_tools("What is the weather in Seattle in Fahrenheit?")
print(response)

Now explore how the agent analyze the question and decide to use only the calculator tool

In [ ]:
# Test with a math question that only needs the calculator tool
# The agent will analyze the question and decide to use only the calculator tool
math_query = "What is 25 * 4 + 18?"
print("=== Agent Chooses Calculator Tool ===")
print(f"Query: {math_query}")
response = agent_with_tools(math_query)
print(f"Response: {response}")

### Direct Tool Invocation

You can also call tools directly without going through the agent conversation. This is useful for testing or when you want to use a specific tool function.

**Specification for direct tool invocation:**
- Use `agent.tool.tool_name()` to call a specific tool directly
- Pass the required parameters as function arguments
- This bypasses the agent's natural language processing and tool selection logic
- Useful for programmatic access to tool functionality

In [ ]:
# Call the calculator tool directly
# Note: Direct tool invocation bypasses the agent's conversation flow.
# Use agent_with_tools.tool.calculator() to call the calculator tool directly
# without the agent deciding which tool to use. This is useful for testing
# specific tools or when you know exactly which tool function you need.
result = agent_with_tools.tool.calculator(expression="2 + 3 * 4")
print(f"Calculator result: {result}")

## Task 2.4: Configure logging

In this task, you set up logging to understand what your agent is doing behind the scenes. This helps you understand how the agent processes requests and uses tools.

The Strands framework uses Python's standard logging module to provide visibility into agent operations. You can configure different log levels to get more or less detail about what your agent is doing.

In [ ]:
import logging
from strands import Agent
import os

# Enable detailed logging to understand what the agent is doing
logging.getLogger("strands").setLevel(logging.INFO)

# Set up logging to write to both console and file
logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    level=logging.INFO,
    handlers=[
        logging.StreamHandler()  # Console output
    ]
)

# Create a logger
logger = logging.getLogger("agent_activity")

# Create an agent with logging enabled
logger.info("Creating new agent with Nova Lite model")
logged_agent = Agent(callback_handler=None,model="amazon.nova-lite-v1:0")

logger.info("Sending message to agent: 'Hello! How are you?'")
response = logged_agent("Hello! How are you?")
print(response)

## Task 2.5: Explore model configuration

In this task, you learn how to configure different models and settings for your agents. You can specify which AI model to use and adjust its behavior with parameters like temperature. Temperature controls how creative or consistent the responses are. Lower temperature values make responses more consistent and predictable, while higher values make them more creative and varied.

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# Create a custom model configuration
custom_model = BedrockModel(
    model_id="amazon.nova-lite-v1:0",
    temperature=0.3  # Lower temperature = more consistent responses
)

# Create an agent with the custom model
custom_agent = Agent(callback_handler=None,model=custom_model)
print("Agent created successfully!")

## Task 2.6: Build a recipe assistant agent

In this task, you create a more practical agent that can help with cooking. This recipe assistant can search the web for recipes and cooking information, showing how agents can be useful in real-world scenarios.

First, install the web search package that your recipe agent needs.

In [ ]:
%pip install ddgs

Create a web search tool that your recipe agent can use to find cooking information and recipes online.

In [ ]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Set up logging
logging.getLogger("strands").setLevel(logging.INFO)

# Create a web search tool
@tool
def websearch(keywords: str, max_results: int = 3) -> str:
    """Search the web for information.
    Args:
        keywords (str): What to search for
        max_results (int): How many results to return
    Returns:
        Search results as text
    """
    try:
        results = DDGS().text(keywords, max_results=max_results)
        return results if results else "No results found."
    except Exception as e:
        return f"Search error: {e}"

print("Web search tool created successfully!")

Now create the recipe assistant agent that uses the web search tool to help with cooking questions.

In [ ]:

# Create the recipe assistant agent
recipe_agent = Agent(callback_handler=None,
    model="amazon.nova-lite-v1:0",
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes and answer cooking questions.
    Use the websearch tool to find recipes and cooking information.""",
    tools=[websearch]
)

print("Recipe assistant agent created successfully!")

Test your recipe assistant by asking it for cooking help. Watch how it uses the web search tool to find current information.

In [ ]:
# Test the recipe assistant
response = recipe_agent("Suggest a simple recipe with chicken and broccoli.")
print(response)

You have successfully created a practical AI agent that can search the web and help with cooking questions. This demonstrates how agents can be useful for real-world tasks.

You have now experimented with the Strands Agents framework, which provides a way to build AI agents that can use tools and take actions. Using this framework, you have learned how agents differ from traditional LLMs by being able to actively use tools to accomplish tasks.

### Try it yourself
- Modify the system prompts to create agents for different use cases.
- Create custom tools for specific tasks your team might need.
- Experiment with different model configurations to understand how they affect agent behavior.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with the **Conclusion**.